# Python Wrapper for CMR
`A python library to interface with CMR - Token Demo`

This demo will show how to request an EDL token from CMR while inside a notebook.

## Practice Demo Safety
Just for this demo, I'm going to create a function that hides some of an EDL token, I don't want anyone to actually see my tokens.

In [ ]:
def safe_token_print(actual_token):
    if actual_token is None:
        print ("no token")
        return
    token_len = len(actual_token)
    keep = int(token_len / 4)
    token_len = len(actual_token)
    strike = "*" * (token_len-(keep*2))
    print (actual_token[:keep] + strike + actual_token[token_len-keep:])

print ("example:")
safe_token_print("012345678909876543210")

## Loading the library 
Do one of the following, but not both

### Choice A) With PIP
Load the library via pip in one of these ways:

* `./runme.sh -p -i` - install from your local copy with the runme script
* `pip3 install https://github.com/nasa/eo-metadata-tools/releases/download/latest-master/eo_metadata_tools_cmr-0.0.1-py3-none-any.whl` - install pre-packaged version from github

### Choice B) Reference Dev Copy
This will not be needed once we have the library working through PIP, but with this, a local checkout can be used. In this case, the command `git clone https://github.com/nasa/eo-metadata-tools` was called in `~/src/project`.

In [ ]:
import os
import sys
sys.path.append(os.path.expanduser('~/src/project/eo-metadata-tools/CMR/python/'))

### Import the library
This should be all you need after we get PIP support. Take care to make sure you install to the same version of python if you have multiple instances.

In [ ]:
import cmr.auth.token as t

import platform
print ("Found in " + platform.python_version())

## Using a token file
In this example we are going to store our password in a file, listed below is how you can specify the file, however the setting is actually the assumed file if no setting is given.

To get a token, generate one using the [URS user generated token page](https://urs.earthdata.nasa.gov/user_tokens). Take that token and store it in the file `~/.cmr_token`.

In [ ]:
options = {"cmr.token.file": "~/.cmr_token"} #this is the default actually
safe_token_print(t.token(t.token_file, options))

options = {} #use no overrides
safe_token_print(t.token(t.token_file, options))

options = {"cmr.token.file": "~/.cmr_token_fake_file"} #this is not the default
safe_token_print(t.token(t.token_file, options))

## Using Keychain on Mac OS X
in this example I am using an already existing password saved securly in keychain. For directions on how to setup a keychain record, see the [README](https://github.com/nasa/eo-metadata-tools/tree/master/CMR/python). Keychain may require a human to type in the password, I have clicked "Allways allow" so we may not see it. When using this method, delete the value in `~/.cmr_token` as having a clear text token defeats the security offered by Keychain. 

In [ ]:
options = {'token.manager.service': 'cmr lib token'} #this is not the default
safe_token_print(t.token(t.token_manager, options))

### Search both at once

In [ ]:
options = {"cmr.token.file": "~/.cmr_token_fake_file", 'token.manager.service': 'cmr lib token'}
safe_token_print(t.token([t.token_manager, t.token_file], options))

## Now search with a token
Do a very basic search in production using a token. Results **may** very based on the permisions of the user for which the token is for.

In [ ]:
import cmr.search.collection as coll

In [ ]:
# Common settings for both queries
params={'keyword':'modis', 'sort_key': 'instrument'}
filters=[coll.collection_core_fields]
configs1={'env': 'uat'}

# For the second query a token will be called from keychain and sent as a Bearer token
token = "Bearer: " + t.token()
configs2=configs1.copy()
configs2['cmr-token'] = token

print(coll.search(params, filters=filters, config=configs1, limit=2))
print('\nvs\n')
safe_token_print(token)
print(coll.search(params, filters=filters, config=configs2, limit=2))

## Bulit in help
I can't remember anything, so here is some built in help which pulls from the python docstring for each function of interest

In [ ]:
print(t.help_text('token_'))

----
The End